In [ ]:
# General packages
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import multiprocessing

# Sci-kit learn packages
from sklearn.metrics import root_mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

# Packages for results/plotting
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import ticker
from shap import summary_plot, TreeExplainer

# Change default font of matplotlib to monospace
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams["font.family"] = "monospace"

In [ ]:
DATASET = 'cmcqrd' # 'usmle', 'bio' or 'cmcqrd'
RESULTS_DATASET = '../data/' + DATASET + '/preprocessed/combined_results'
TARGET_LABEL_COL_NAME = 'Correct_Answer_Rate' # "Correct_Answer_Rate" or "difficulty"(only for cmcqrd) or "Response_Time"(for usmle only)
REPETITIONS = 10  # Number of repetitions for each experiment, average will be taken
# Number of cores to use for sklearn's n_jobs parameter, whenever possible
NUM_OF_CORES_TO_USE = multiprocessing.cpu_count()
print("Using ", NUM_OF_CORES_TO_USE, " cores.")
# TF-IDF threshold for feature selection
TFIDF_THRESHOLD = 0.0007  # best found: 0.0007, higher->fewer features
# Number of most important features to print for the random forest model
NUMBER_OF_IMPORTANT_FEATURES_TO_PRINT = 10
# Define the feature columns from the other models, that contain other uncertainty features
MODEL_NAMES = ['phi3_5-chat', 'Llama3_2-3b-chat', 'Qwen2_5-3b-chat', 'Llama3_1-8b-chat', 'Qwen2_5-14b-chat', 'Qwen2_5-32b-chat', 'Yi-34b-chat', 'Qwen2_5-72b-chat', 'Llama3_1-70b-chat']

# Which experiments to run:
FULL_PRECISION_MODELS = True
CHOICE_SIMILARITY_EXPERIMENTS = True

# Prepare export folders/make sure they are there. 
if not os.path.exists('plots'):
    os.makedirs('plots')
if not os.path.exists('plots/' + DATASET):
    os.makedirs('plots/' + DATASET)
if not os.path.exists('plots/' + DATASET + '/SHAP_summary'):
    os.makedirs('plots/' + DATASET + '/SHAP_summary')

## Load the data and splits

In [ ]:
train = None
test = None

if FULL_PRECISION_MODELS:
    train = pd.read_csv(RESULTS_DATASET + '_fp_train_set.csv')
    test = pd.read_csv(RESULTS_DATASET + '_fp_test_set.csv', index_col=0)
else:
    train = pd.read_csv(RESULTS_DATASET + '_train_set.csv')
    test = pd.read_csv(RESULTS_DATASET + '_test_set.csv', index_col=0)


train_with_linguistic = pd.read_csv('../data/' + DATASET + '/with_ling_features/' + 'train.csv')
test_with_linguistic = pd.read_csv('../data/' + DATASET + '/with_ling_features/' + 'test.csv', index_col=0)

# Size per split
print("Train size: ", len(train))
print("Test size: ", len(test))
train.head()

## Create TF-IDF features from text

In [ ]:
# Convert to TF-IDF scores
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),  # Use 1-grams and 2-grams.
    # Ignore terms that appear in less than 0.1% of the documents.
    min_df=0.001,
    # Ignore terms that appear in more than 75% of documents.
    max_df=0.75,
    max_features=1000,  # Use only the top 1000 most frequent words.
    stop_words='english'
)

textTrain = vectorizer.fit_transform(train['question_with_options']).toarray()
textTest = vectorizer.transform(test['question_with_options']).toarray()

textTrain = pd.DataFrame(
    textTrain,
    columns=['"' + w + '"' for w in vectorizer.get_feature_names_out()],
    index=train.index
)

textTest = pd.DataFrame(
    textTest,
    columns=['"' + w + '"' for w in vectorizer.get_feature_names_out()],
    index=test.index
)

In [ ]:
# Adjust threshold as needed, the lower the threshold, the more features will be selected.
selector = VarianceThreshold(threshold=TFIDF_THRESHOLD)
textTrain_selected = selector.fit_transform(textTrain)
textTrain_selected = pd.DataFrame(textTrain_selected,
                                  columns=textTrain.columns[selector.get_support(
                                  )],
                                  index=textTrain.index)

# Apply to test set
textTest_selected = selector.transform(textTest)
textTest_selected = pd.DataFrame(textTest_selected,
                                 columns=textTrain.columns[selector.get_support(
                                 )],
                                 index=textTest.index)

print(textTrain_selected.shape)
textTrain = textTrain_selected
textTest = textTest_selected

___
# Experiments

In [ ]:
GLOBAL_ALL_RESULTS = {} # Store all results here

def test_random_forest(features_train, target_train, features_test, target_test, description):
    """Runs a Random Forest model on the given data and returns the RMSE and the top features. Average of REPETITIONS is taken."""
    feature_importances_sum = None

    all_rmses_for_model = []

    # Initialize a sum array for feature importances
    feature_importances_sum = np.zeros(features_train.shape[1])  # Number of features in the dataset)
    shap_values = []
    for repetition in tqdm(range(REPETITIONS)):
        model = RandomForestRegressor(n_jobs=NUM_OF_CORES_TO_USE) # Re-initialize the model every time
        # Fit the random forest. model.fit resets the model every time so it doesn't remember the previous fit.
        model.fit(features_train, target_train)
        # Predict the target values
        predictions = model.predict(features_test)
        rmse = root_mean_squared_error(predictions, target_test)  # Calculate RMSE
        all_rmses_for_model.append(rmse)
        # Collect feature importances
        feature_importances_sum += model.feature_importances_
        ### SHAP
        explainer = TreeExplainer(model, approximate=True)
        shap_values_of_repetition = explainer(features_train)
        shap_values.append(shap_values_of_repetition)


    # Calculate statistics for the model
    average_rmse_for_model = float(np.mean(all_rmses_for_model)) # Mean RMSE
    std_dev_rmse = float(np.std(all_rmses_for_model, ddof=1))  # Sample standard deviation
    std_error_rmse = float(std_dev_rmse / np.sqrt(REPETITIONS))  # Standard error

    # Store a summary of the results
    rmse_results_summary = {
        'rmse': round(average_rmse_for_model, 4),
        'std_dev': round(std_dev_rmse, 4),
        'std_error': round(std_error_rmse, 4)
    }

    # Print the top NUMBER_OF_IMPORTANT_FEATURES_TO_PRINT important features for Random Forest
    # Calculate the average feature importances across all repetitions
    avg_feature_importances = feature_importances_sum / REPETITIONS
    # Get the indices of the top NUMBER_OF_IMPORTANT_FEATURES_TO_PRINT most important features
    indices = np.argsort(avg_feature_importances)[-NUMBER_OF_IMPORTANT_FEATURES_TO_PRINT:][::-1]
    top_features_and_importances = [(features_train.columns[i], float(round(avg_feature_importances[i], 4))) for i in indices]
    
    # Average SHAP values over repetitions
    shap_values = np.array([shap_values[i].values for i in range(REPETITIONS)])
    shap_values = np.mean(shap_values, axis=0)
    
    # Summary plot:
    summary_plot(shap_values, features_train, max_display=NUMBER_OF_IMPORTANT_FEATURES_TO_PRINT, plot_size=[12, 7], show=False)
    plt.suptitle('Effect of Top Features on Predicting Student Success \n(' + description + ')', fontsize=20, x=0.5, y=1.1)
    # change font size of the ticks
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=16)
    plt.xlabel('Impact on Random Forest Prediction', fontsize=18)
    ax = plt.gca()  # Get current axis
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{x:.2g}"))

    
    plt.savefig(f'plots/{DATASET}/SHAP_summary/{DATASET}_shap_summary_' + description + '.png', bbox_inches='tight', dpi=300)
    plt.close()    
    
    return rmse_results_summary, top_features_and_importances


def retrieve_models_uncertainties_col_names(metric_names):
    uncertainty_feature_columns = []
    for metric in metric_names:
        for model in MODEL_NAMES:
            uncertainty_feature_columns.append(f'{metric}_{model}')
    return uncertainty_feature_columns

## Dummy Regressor (baseline)

In [ ]:
# Concatenate the features
features_train = pd.concat([textTrain], axis=1)
target_train = train[TARGET_LABEL_COL_NAME]
features_test = pd.concat([textTest], axis=1)
target_test = test[TARGET_LABEL_COL_NAME]

# Fit the dummy regressor
dummy_regressor = DummyRegressor(strategy="mean")
dummy_regressor.fit(features_train, target_train)

# Predict the target values
dummy_regressor_predictions = dummy_regressor.predict(features_test)
rmse = float(root_mean_squared_error(target_test, dummy_regressor_predictions)) # float converts to normal float, not numpy float

GLOBAL_ALL_RESULTS['DummyRegressor'] = ({'rmse': round(rmse, 4), 'std_dev': 0, 'std_error': 0}, [], None)

## Only TF-IDF features

In [ ]:
features_train, target_train, features_test, target_test = None, None, None, None
# Concatenate the features
features_train = pd.concat([textTrain], axis=1)
target_train = train[TARGET_LABEL_COL_NAME]
features_test = pd.concat([textTest], axis=1)
target_test = test[TARGET_LABEL_COL_NAME]

description = "Only TF-IDF Features"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Only First Token Probability (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability'])

# Concatenate the features
ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)

features_train = pd.concat([ensemble_cols_train], axis=1)
features_test = pd.concat([ensemble_cols_test], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "All Models' 1st Token Probabilities"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Only Choice Order Probability  (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['order_probability'])

# Concatenate the features
ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)

features_train = pd.concat([ensemble_cols_train], axis=1)
features_test = pd.concat([ensemble_cols_test], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "All Models' Choice Order Probabilities"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## First Token Probability and Choice Order Probability  (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

# Concatenate the features
ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)

features_train = pd.concat([ensemble_cols_train], axis=1)
features_test = pd.concat([ensemble_cols_test], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "All Models' Uncertainties"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## TF-IDF & 1st Token Probability  (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability'])

ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
features_train = pd.concat([ensemble_cols_train, textTrain], axis=1)

ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
features_test = pd.concat([ensemble_cols_test, textTest], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "TF-IDF and All Models' 1st Token Probabilities"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## TF-IDF & Choice Order Probability  (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['order_probability'])

ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
features_train = pd.concat([ensemble_cols_train, textTrain], axis=1)

ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
features_test = pd.concat([ensemble_cols_test, textTest], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "TF-IDF and All Models' Choice Order Probabilities"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## TF-IDF & Both Uncertainties (all models)

In [ ]:
uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
features_train = pd.concat([ensemble_cols_train, textTrain], axis=1)

ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
features_test = pd.concat([ensemble_cols_test, textTest], axis=1)

target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

description = "TF-IDF and All Models' Uncertainties"
GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

___
# Choice Similarity Experiments

## Only Choices similarity

In [ ]:
if CHOICE_SIMILARITY_EXPERIMENTS:
    similarity_to_use = "choices_similarity" # choices_similarity or choices_similarity_clinical !!!
    # Concatenate the features
    features_train = pd.concat([
        train[similarity_to_use],
    ], axis=1)
    features_test = pd.concat([
        test[similarity_to_use],
    ], axis=1)

    target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

    description = "Only " + similarity_to_use
    GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Text, both uncertainties & Choices similarity (all models)

In [ ]:
if CHOICE_SIMILARITY_EXPERIMENTS:
    uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

    # Concatenate the features
    ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
    features_train = pd.concat([ensemble_cols_train, train['choices_similarity'] , textTrain], axis=1)

    ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
    features_test = pd.concat([ensemble_cols_test, test['choices_similarity'] , textTest], axis=1)
    target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

    description = "TF-IDF and All Models' Uncertainties and Choices Similarity"
    GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Text, both uncertainties and Medical Choice Similarity

In [ ]:
if CHOICE_SIMILARITY_EXPERIMENTS:
    uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

    # Concatenate the features
    ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
    features_train = pd.concat([ensemble_cols_train, train['choices_similarity_clinical'] , textTrain], axis=1)

    ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
    features_test = pd.concat([ensemble_cols_test, test['choices_similarity_clinical'] , textTest], axis=1)
    target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

    description = "TF-IDF and All Models' Uncertainties and (clinical) Choices Similarity"
    GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Text, both uncertainties and both Choice Similarities

In [ ]:
if CHOICE_SIMILARITY_EXPERIMENTS:
    uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

    # Concatenate the features
    ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
    features_train = pd.concat([ensemble_cols_train, train['choices_similarity_clinical'], train['choices_similarity'] , textTrain], axis=1)

    ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
    features_test = pd.concat([ensemble_cols_test, test['choices_similarity_clinical'],test['choices_similarity'] , textTest], axis=1)
    target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

    description = "TF-IDF and All Models' Uncertainties and both Choices Similarity"
    GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

## Text, both uncertainties, both Choice Similarities and Linguistic Features

In [ ]:
if CHOICE_SIMILARITY_EXPERIMENTS:
    linguistic_features= ["Word_Count", "Word_Count_No_stop_words", "Avg_Word_Length", "Sentence_Count", "Avg_Sent_Length_in_Words", "Noun_Count", "Verb_Count", "Adjective_Count", "Adverb_Count", "Number_of_NPs", "Number_of_PPs", "Number_of_VPs", "Temporal_Connectives_Count", "Causal_Connectives_Count", "Exemplifying_Connectives_Count", "Additive_Connectives_Count", "Contrastive_Connectives_Count"]
    
    uncertainty_feature_columns = retrieve_models_uncertainties_col_names(['first_token_probability', 'order_probability'])

    # Concatenate the features
    ensemble_cols_train = pd.concat([train[col] for col in uncertainty_feature_columns], axis=1)
    features_train = pd.concat([ensemble_cols_train, train['choices_similarity'], train['choices_similarity_clinical'], train_with_linguistic[linguistic_features], textTrain], axis=1)

    ensemble_cols_test = pd.concat([test[col] for col in uncertainty_feature_columns], axis=1)
    features_test = pd.concat([ensemble_cols_test, test['choices_similarity'], test['choices_similarity_clinical'], test_with_linguistic[linguistic_features], textTest], axis=1)
    target_train, target_test = train[TARGET_LABEL_COL_NAME], test[TARGET_LABEL_COL_NAME]

    description = "TF-IDF, All Models' Uncertainties, All Choices Similarities \n and Linguistic Features"
    GLOBAL_ALL_RESULTS[description] = test_random_forest(features_train, target_train, features_test, target_test, description)

___
# Results RMSE Overview and Top Features

In [ ]:
# Now we filter results to only the ones we are interested in
checkboxes = [widgets.Checkbox(value=True, description=label, layout=widgets.Layout(
    width='1000px')) for label in GLOBAL_ALL_RESULTS]
output = widgets.VBox(children=checkboxes)
display(output)

In [ ]:
selected_keys = []
for i in range(0, len(checkboxes)):
    if checkboxes[i].value == True:
        selected_keys = selected_keys + [checkboxes[i].description]

for key in selected_keys:
    print(key)
    print("RMSE: ", GLOBAL_ALL_RESULTS[key][0]['rmse'], "±", GLOBAL_ALL_RESULTS[key][0]['std_error'], " (STDEV: ", GLOBAL_ALL_RESULTS[key][0]['std_dev'], ")")
    print("--------------------")
    if key == 'DummyRegressor':
        # there are no features to plot
        continue
    ### Plotting ###
    # Set the font
    plt.rcParams["font.family"] = "monospace"
    
    # Create the main plot
    fig, ax = plt.subplots(figsize=(10, 5))
    GLOBAL_ALL_RESULTS[key][1].sort(key=lambda x: x[1]) # descending order by importance
    ax.barh([x[0] for x in GLOBAL_ALL_RESULTS[key][1]], [x[1] for x in GLOBAL_ALL_RESULTS[key][1]], color='#478058')
    
    # Set title and labels
    title_text = 'Feature Contributions to Model Performance\n' + '(' + key + ')' # title text
    fig.suptitle(title_text, ha='center', fontsize=20, y=1.01) # set title
    ax.set_xlabel('Feature Importance', fontsize=18) # x label
    # set y ticks manually, so they are within the barplot using plt.text
    max_importance = max([x[1] for x in GLOBAL_ALL_RESULTS[key][1]])
    for i, (feature, importance) in enumerate(GLOBAL_ALL_RESULTS[key][1]):
        if importance > max_importance/2:
            text_length = len(feature) * 0.002 # This is very hacky: manually tune this
            ax.text(0.01, i, feature, ha='left', va='center', fontsize=10, color='white') # aligned to the right of the bar
        else:
            ax.text(0.0001+importance, i, feature, ha='left', va='center', fontsize=10, color='black') # might need to adjust the 0.0001

    ax.set_yticks([])
    plt.show()
    fig.savefig(f'plots/{DATASET}/{key}.png', dpi=200, bbox_inches='tight')
